In [1]:
import os, glob, sys

In [2]:
%env CITYSCAPES_DATASET = /home/rvygon/data/

env: CITYSCAPES_DATASET=/home/rvygon/data/


In [3]:
from ImportUtil import *
initTrain()
X_train, y_train = importBatch(500,0)

In [6]:
imshow(y_train[1])

/opt/conda/anaconda3/lib/python3.6/site-packages/skimage/io/_plugins/matplotlib_plugin.py:51: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  out_of_range_float = (np.issubdtype(image.dtype, np.float) and


In [7]:
print(y_train[0][50])

[255 255 255 ... 255 255 255]


In [ ]:
run cityscapesscripts/preparation/createTrainIdInstanceImgs


/home/rvygon/data/gtFine/*/*/*_gt*_polygons.json
Processing 5000 annotation files
Progress: 5.14 %                      